In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

df = pd.read_csv('data_uncleaned.csv')

/Users/JPhilleo/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,2,3,4,6,7,8,9,10,11,12,13,15,16,17,19,21,24,25,26,27,28,29,31,33,35,36,37,39,41,42,43,44,45,46,49,50,51,52,53,54,55,56,57,58,60,64,65,66,68,71,72,73,74,75,76,77,78,79,80,81,82,83,85,86,89,90,92,93,96,98,99,101,102,103,104,105,107,109,111,113,114,115,116,117,118,119,121,123,125,126,128,129,131,132,133,134,136,138,140,141,143,145,146,147,148,149,150,151,152,154,155,156,157,158,159,160,161,162,164,165,166,168,169,172,173,174,176,177,179,180,181,182,184,185,186,187,188,189,190,191,192,193,196,197,200,201,202,204,207,209,210,211,217,218,220,221,222,223,225,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,256,258,259,261,262,263,264,267,269,270,271,272,273,274,275,277,278,279,280,281,282,283,284,285,288,290,291,296,298,299,300,302,303,304,306,307,308,309,310,311,313,314,315,316,317,318,319,320,321,323,324,325,326,329,330,3

In [2]:
df.shape

(8643, 515)

In [3]:
df.head()

,365chess player ID,ACM Digital Library author ID,ADK member ID,AGORHA person/institution ID,AcademiaNet,Academic Tree ID,AllMovie artist ID,AllMusic artist ID,AlloCiné person ID,Art UK artist ID,...,topic's main template,unmarried partner,uses,video,warheroes.ru ID,winner,work location,zbMATH author ID,ČSFD person ID,اللللل
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,['Paris'],['boscovich.ruggiero-giuseppe'],NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['deahna.f'],NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns.values

array(['365chess player ID', 'ACM Digital Library author ID',
       'ADK member ID', 'AGORHA person/institution ID', 'AcademiaNet',
       'Academic Tree ID', 'AllMovie artist ID', 'AllMusic artist ID',
       'AlloCiné person ID', 'Art UK artist ID', 'Artnet artist ID',
       'Artsy artist ID', 'Australian Dictionary of Biography ID',
       'Auñamendi ID', 'BAV ID', 'BBF ID', 'BDCYL authority ID',
       'BHL creator ID', 'BIBSYS ID', 'BMLO', 'BNA authority ID', 'BNC ID',
       'BNE ID', 'BVMC person ID', 'Babelio author ID',
       'Bait La Zemer Ha-Ivri artist ID',
       'Baltisches Biographisches Lexikon digital ID', 'BanQ author ID',
       'Benezit ID', 'BiblioNet author ID', 'Biografisch Portaal number',
       'Bloomberg person ID', 'BnF ID', 'Bridgeman artist ID',
       'British Museum person-institution', 'British Museum thesaurus ID',
       'C-SPAN person ID', 'CALIS ID', 'CANTIC-ID', 'CBDB ID', 'CERL ID',
       'CESAR person ID', 'CLARA-ID', 'CONICET person ID', 'CO

In [5]:
df.dtypes

365chess player ID                                               object
ACM Digital Library author ID                                    object
ADK member ID                                                    object
AGORHA person/institution ID                                     object
AcademiaNet                                                      object
Academic Tree ID                                                 object
AllMovie artist ID                                               object
AllMusic artist ID                                               object
AlloCiné person ID                                               object
Art UK artist ID                                                 object
Artnet artist ID                                                 object
Artsy artist ID                                                  object
Australian Dictionary of Biography ID                            object
Auñamendi ID                                                    

In [6]:
# discard columns with less than ten entries
for column in df.columns:
    if df[column].count() < 10:
        df = df.drop(column, axis=1)

print(df.shape)

(8643, 231)


In [7]:
df.count().sort_values(ascending=False)

mathematicians                                      8643
links                                               8643
instance of                                         8628
sex or gender                                       8575
occupation                                          8285
date of birth                                       7665
given name                                          7313
country of citizenship                              6879
VIAF ID                                             6476
place of birth                                      5588
Mathematics Genealogy Project ID                    5168
date of death                                       4908
SUDOC authorities                                   4826
Library of Congress authority identifier            4790
educated at                                         4727
GND ID                                              4360
ISNI                                                4326
BnF ID                         

In [8]:
# for easier analysis, sort df columns by number of entries
columns = df.count().sort_values(ascending=False).index
df = df[columns]

In [9]:
# functions used do not work with NaNs, so convert NaNs to empty strings
for i in df.columns:
    df[i][df[i].isnull()] = ''

print('number of non-humans in df: ', df[~((df['instance of'].str.contains("'human'")) | (df['instance of'].str.contains("'Q5'")))].shape[0])
df[~((df['instance of'].str.contains("'human'")) | (df['instance of'].str.contains("'Q5'")))]

number of non-humans in df:  46


,mathematicians,links,instance of,sex or gender,occupation,date of birth,given name,country of citizenship,VIAF ID,place of birth,...,Facebook profile ID,military rank,National Gallery of Art artist ID,Les Archives du Spectacle ID,Dictionary of Ulster Biography ID,Dictionary of Welsh Biography ID,member of sports team,WomenWriters ID,NILF author id,365chess player ID
337,Oxford Calculators,https://www.wikidata.org/wiki/Special:EntityPa...,['group of humans'],,,,,,,,...,,,,,,,,,,
338,Ronald Fisher bibliography,https://www.wikidata.org/wiki/Special:EntityPa...,"['personal bibliography', 'Ronald Fisher']",,,,,,,,...,,,,,,,,,,
490,Hindu and Buddhist contribution to science in ...,https://www.wikidata.org/wiki/Special:EntityPa...,,,,,,,,,...,,,,,,,,,,
589,AWM/MAA Falconer Lecturer,https://www.wikidata.org/wiki/Special:EntityPa...,['award'],,,,,,,,...,,,,,,,,,,
648,Fields Medal,https://www.wikidata.org/wiki/Special:EntityPa...,"['science award', 'mathematics']",,,,,,,,...,,,,,,,,,,
958,Malba Tahan,https://www.wikidata.org/wiki/Special:EntityPa...,['fictional human'],['male'],,,,,,,...,,,,,,,,,,
1059,Founders of statistics,https://www.wikidata.org/wiki/Special:EntityPa...,['Wikimedia list article'],,,,,,,,...,,,,,,,,,,
1151,Banū Mūsā,https://www.wikidata.org/wiki/Special:EntityPa...,['sibling group'],['male'],"['mathematician', 'engineer', 'scientist', 'as...","['9. century', 'statement with Gregorian date ...",,['Iraq'],['46045337'],['Baghdad'],...,,,,,,,,,,
1238,John Rainwater,https://www.wikidata.org/wiki/Special:EntityPa...,['fictional character'],['male'],['mathematician'],,['John'],,,,...,,,,,,,,,,
1278,Count von Count,https://www.wikidata.org/wiki/Special:EntityPa...,['vampire in a work of fiction'],['male'],,,,,,,...,,,,,,,,,,


In [10]:
# # these mathematicians were not classified as humans, but seem like they should be
# # for some reason, my dataframe does not recognize them as existing... :(
# missing = ['Igor Mezić', 'John Wood (professor)', 'Otfried Cheong', 'Liu Gang']

# for i in missing:
#     if i in df.mathematicians:
#         print(i)
# #     df['instance of'][df['mathematicians'] == i] = ['human']

# # print(df[~((df['instance of'].str.contains("'human'")) | (df['instance of'].str.contains("'Q5'")))].shape)

In [11]:
print('total (old):', df.shape[0])
df = df[(df['instance of'].str.contains("'human'")) | (df['instance of'].str.contains("'Q5'")) | (df['instance of'].str.contains("'sibling'"))]
print('human (new):', df.shape[0])

total (old): 8643
human (new): 8597


In [12]:
for i in df['instance of'].unique():
    print(i)

['human']
['human', 'Russian Wikipedia']
['human', 'twin']
['Q5']
['twin', 'human']
['human', 'English Wikipedia']
['human', 'data.bnf.fr', '10 October 2015', 'http://data.bnf.fr/ark:/12148/cb119176085']
['human', 'Russian Wikipedia', 'data.bnf.fr', '10 October 2015', 'http://data.bnf.fr/ark:/12148/cb118976048']
['eunuch', 'human']
['human', 'male']
['human', 'emeritus']


In [13]:
df['sex or gender'][~((df['sex or gender'].str.contains('male')) | (df['sex or gender'].str.contains('female')))].unique()

array([''], dtype=object)

In [14]:
def cleanText(df, column):
    df[column] = df[column].str.replace("'", "")
    df[column] = df[column].str.replace("[", "")
    df[column] = df[column].str.replace("]", "")
    return df

def cleanList(df, column):
    df = cleanText(df, column)
    df[column] = df[column].str.split(', ')
    return df

In [15]:
df = cleanText(df, 'given name')
df = cleanText(df, 'place of birth')

In [16]:
approximated = []

def cleanDate(df, column, name):
    # clean date text
    df = cleanText(df, column)
    df[column] = df[column].str.replace("Gregorian", "")
    df[column] = df[column].str.replace("Julian", "")
    df[column] = df[column].apply(lambda x: x.split(',', 1)[0])
    
    # create an "approximated" column to recognize inexact values
    approx = 'approx. ' + column
    approximated.append(approx)
    df[approx] = df[column].str.contains('century', na=False)
    df[column][(df[approx]) & (df[column].str[1] != '.')] = df[column][(df[approx]) & (df[column].str[1] != '.')].apply(lambda x: str(int(x[:2])-1) + '00')
    df[column][(df[approx]) & (df[column].str[1] == '.')] = df[column][(df[approx]) & (df[column].str[1] =='.')].apply(lambda x: str(int(x[0])-1) + '00')

    # break date into separate year, month, day columns
    year = 'year of ' + name
    month = 'month of ' + name
    day = 'day of ' + name
    
    df[day], df[month], df[year] = df[column].str.split(' ', 2).str
    
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
              'August', 'September', 'October', 'November', 'December']

    # fix dates with 'year' in 'day' column
    df[year][df[day].str.len() > 2] = df[day][df[day].str.len() > 2]
    df[day][df[day].str.len() > 2] = ''
    
    df[year][df[day].apply(pd.to_numeric, args=('coerce',)) > 31] = df[day]
    df[day][df[day].apply(pd.to_numeric, args=('coerce',)) > 31] = ''
    
    df[year][df[day].str[-1:] == 's'] = df[day].str[:-1]
    df[day][df[day].str[-1:] == 's'] = ''
    
    # fix dates with 'month' in 'day' column
    df[year][df[day].isin(months)] = df[month][df[day].isin(months)]
    df[month][df[day].isin(months)] = df[day][df[day].isin(months)]
    df[day][df[day].isin(months)] = ''

    for i in df.columns:
        df[i][df[i].isnull()] = ''

    # fix dates with flipped year-month
    condition = df[month].str.isnumeric()
    df[day][condition] = df[year][condition]
    df[year][condition] = df[month][condition]
    df[month][condition] = df[day][condition]
    df[day][condition] = ''

    # convert BCE to negative values
    df[year][df[month] == 'BCE'] = '-' + df[year][df[month] == 'BCE']
    df[month][df[month] == 'BCE'] = ''
    
    # remove date with 'millennium' in it
    df = df[df[column].str.contains('millennium') == False]
    
    return df

df = cleanDate(df, 'date of birth', 'birth')
df = cleanDate(df, 'date of death', 'death')
df

/Users/JPhilleo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/JPhilleo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/JPhilleo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/JPhilleo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying

,mathematicians,links,instance of,sex or gender,occupation,date of birth,given name,country of citizenship,VIAF ID,place of birth,...,NILF author id,365chess player ID,approx. date of birth,day of birth,month of birth,year of birth,approx. date of death,day of death,month of death,year of death
0,Roger Joseph Boscovich,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],"['physicist', 'astronomer', 'mathematician', '...",18 May 1711,Ruggero,['Republic of Ragusa'],['36938235'],"Dubrovnik, Republic of Ragusa",...,,,False,18,May,1711,False,13,February,1787
1,Emma Previato,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['female'],['mathematician'],1952,Emma,"['United States of America', 'Italy']",['71642559'],Badia Polesine,...,,,False,,,1952,False,,,
2,Feodor Deahna,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],['mathematician'],1815,Feodor,,['35350194'],,...,,,False,,,1815,False,,,1844
3,Denis Henrion,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],"['publisher', 'mathematician']",1500,Denis,['France'],['10077255'],,...,,,True,,,1500,False,,,1640
4,Henri Delannoy,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],"['mathematician', 'historian', 'military perso...",28 September 1833,"Henri, Auguste",['France'],['4920573'],Bourbonne-les-Bains,...,,,False,28,September,1833,False,5,February,1915
5,John Hymers,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],['mathematician'],20 July 1803,John,,['37277653'],Ormesby,...,,,False,20,July,1803,False,7,April,1887
6,Alexander Grothendieck,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],"['mathematician', 'university teacher']",28 March 1928,Alexander,"['France', '1971', '1971', 'statelessness', 's...",['12381506'],Berlin,...,,,False,28,March,1928,False,13,November,2014
7,Theodore James Courant,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],['mathematician'],,Theodore,['United States of America'],['41001596'],,...,,,False,,,,False,,,
8,Ljubomir Chakaloff,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],['mathematician'],18 February 1886,Ljubomir,,['1383716'],,...,,,False,18,February,1886,False,11,September,1963
9,Earnshaw Cook,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],,28 March 1900,,,,,...,,,False,28,March,1900,False,11,November,1987


In [17]:
print(sorted(set(df['day of birth'])))
print()
print()
print(sorted(set(df['month of birth'])))
print()
print()
print(sorted(set(df['year of birth'])))

['', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '4', '5', '6', '7', '8', '9']


['', 'April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September']


['', '-135', '-150', '-160', '-1680', '-190', '-200', '-240', '-250', '-262', '-276', '-280', '-280s', '-281', '-284', '-287', '-310s', '-340', '-350', '-360', '-370', '-380', '-383', '-390', '-396', '-408', '-410s', '-428', '-430', '-443', '-460s', '-465', '-470', '-479', '-480', '-490', '-500', '-585', '-586', '-600', '-623', '-750', '-78', '000', '100', '1000', '1010', '1013', '1019', '1020', '1029', '1031', '1048', '1060', '1070', '1080', '1089', '1098', '1100', '1101', '1110', '1114', '1120', '1130', '1135', '1136', '1175', '1187', '1192', '1195', '1197', '1200', '1201', '1203', '1210', '1213', '1230', '1231', '1232', '1238', '1249', '1250', '1256', '1267', '1282', '1288', 

In [18]:
df[df['day of death'] == '1.']

,mathematicians,links,instance of,sex or gender,occupation,date of birth,given name,country of citizenship,VIAF ID,place of birth,...,NILF author id,365chess player ID,approx. date of birth,day of birth,month of birth,year of birth,approx. date of death,day of death,month of death,year of death


In [19]:
print(sorted(set(df['day of death'])))
print()
print()
print(sorted(set(df['month of death'])))
print()
print()
print(sorted(set(df['year of death'])))

['', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '4', '5', '6', '7', '8', '9']


['', 'April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September']


['', '-100', '-120', '-140', '-1620', '-180', '-190', '-194', '-208', '-212', '-220s', '-221', '-230s', '-278', '-290', '-300', '-314', '-320', '-321', '-347', '-355', '-360s', '-369', '-37', '-370', '-390', '-399', '-409', '-410', '-411', '-425', '-428', '-460', '-500', '-51', '-525', '-546', '-690', '-75', '000', '100', '1000', '1003', '1009', '1022', '1024', '1026', '1029', '1036', '1037', '1040', '1048', '1050', '1054', '1066', '1070', '1075', '1084', '1087', '1095', '1100', '1101', '1130', '1131', '1136', '1145', '1150', '1152', '1157', '1169', '1172', '1180', '1185', '1196', '120', '1200', '1204', '1206', '1213', '1225', '1228', '1232', '1240', '1256', '1260', '1261', '126

In [20]:
occupations = list(set([element for row in cleanList(df, 'occupation').occupation for element in row]))
occupations

['',
 'On the history of logarithms : Bürgi',
 'editor',
 'newspaper editor',
 'anthropologist',
 'full professor',
 'philosopher of language',
 'activist',
 'civil engineer',
 'engineer of roads and bridges',
 'political prisoner',
 'engineer',
 'Christian theology',
 'neuroscientist',
 'algebraic geometry',
 'street artist',
 'singer',
 'lexicographer',
 'opinion journalist',
 'blogger',
 'International School for Advanced Studies',
 'radiologist',
 'translator',
 'naturalist',
 'cryptographer',
 'Russian Wikipedia',
 'Director of Research at CNRS',
 '2000',
 '1981',
 'jazz musician',
 'programmer',
 'toponymist',
 'art collector',
 'nun',
 'juggler',
 'Catholic priest',
 'Q16267607',
 'badminton player',
 'basketball player',
 'biochemist',
 'stage magician',
 'sport cyclist',
 'geomatics engineering',
 'orator',
 'mathematician',
 'philosopher of science',
 'biologist',
 'cricketer',
 'science writer',
 'criminologist',
 'psephologist',
 'horticulturist',
 'metallurgist',
 'model',

In [21]:
def makeDummies(df, column):
    column_list = list(set([element for row in cleanList(df, column)[column] for element in row]))

    list_dummies = pd.DataFrame(columns=column_list[1:], index=df.index)
    df['list_dummies'] = [', '.join(i) for i in df[column]]

    for column in list_dummies.columns:
        for row in range(len(list_dummies)):
            if column in df['list_dummies'].iloc[row]:
                list_dummies[column].iloc[row] = 1
            else:
                list_dummies[column].iloc[row] = 0

    df = pd.concat([df, list_dummies], axis=1)
    df = df.drop(['list_dummies'], axis=1)

    return df

# df = makeDummies(df, 'occupation')
# df = makeDummies(df, 'country of citizenship')
# df = makeDummies(df, 'educated at')
# df = makeDummies(df, 'employer')
# df = makeDummies(df, 'member of')
# df = makeDummies(df, 'field of work')

df.head()

,mathematicians,links,instance of,sex or gender,occupation,date of birth,given name,country of citizenship,VIAF ID,place of birth,...,NILF author id,365chess player ID,approx. date of birth,day of birth,month of birth,year of birth,approx. date of death,day of death,month of death,year of death
0,Roger Joseph Boscovich,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],"[physicist, astronomer, mathematician, philoso...",18 May 1711,Ruggero,['Republic of Ragusa'],['36938235'],"Dubrovnik, Republic of Ragusa",...,,,False,18,May,1711,False,13,February,1787
1,Emma Previato,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['female'],[mathematician],1952,Emma,"['United States of America', 'Italy']",['71642559'],Badia Polesine,...,,,False,,,1952,False,,,
2,Feodor Deahna,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],[mathematician],1815,Feodor,,['35350194'],,...,,,False,,,1815,False,,,1844
3,Denis Henrion,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],"[publisher, mathematician]",1500,Denis,['France'],['10077255'],,...,,,True,,,1500,False,,,1640
4,Henri Delannoy,https://www.wikidata.org/wiki/Special:EntityPa...,['human'],['male'],"[mathematician, historian, military personnel]",28 September 1833,"Henri, Auguste",['France'],['4920573'],Bourbonne-les-Bains,...,,,False,28,September,1833,False,5,February,1915


In [22]:
feautres_of_interest = ['mathematicians', 'occupation', 'country of citizenship',
                        'place of birth', 'date of death', 'educated at', 'employer',
                        'place of death', 'member of', 'employer', 'doctoral advisor',
                        'languages spoken, written or signed', 'academic degree',
                        'doctoral student', 'manner of death', 'position held',
                        'field of work', 'award received','Erdős number', 'instance of',
                        'sex or gender',  'approx. date of birth', 'day of birth',
                        'month of birth', 'year of birth', 'approx. date of death',
                        'day of death', 'month of death', 'year of death']

df = df[feautres_of_interest]

In [23]:
df.shape

(8596, 29)

In [24]:
df.to_csv('data_cleaned.csv', index=False)